In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns
import xgboost as xgb
import os
import gc
gc.enable()
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import warnings
warnings.filterwarnings('ignore')
import urllib        #for url stuff
#import gmaps       #for using google maps to visulalize places on maps
import re            #for processing regular expressions
import datetime      #for datetime operations
import calendar      #for calendar for datetime operations
import time          #to get the system time
import scipy         #for other dependancies
from sklearn.cluster import KMeans # for doing K-means clustering
from haversine import haversine # for calculating haversine distance
import math          #for basic maths operations
import seaborn as sns #for making plots
import matplotlib.pyplot as plt # for plotting
import os  # for os commands
from scipy.misc import imread, imresize, imsave  # for plots 
import plotly.graph_objs as go
import plotly
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_notebook, show
from IPython.display import HTML
from matplotlib.pyplot import *
from matplotlib import cm
from matplotlib import animation
import io
import base64
output_notebook()
plotly.offline.init_notebook_mode() # run at the start of every ipython notebook

In [ ]:
np.random.seed(1987)
N = 100000 # number of sample rows in plots
t0 = dt.datetime.now()
train = pd.read_csv('../input/ride-predict/Train (2).csv')
test = pd.read_csv('../input/ride-predict/Test.csv')
riders = pd.read_csv('../input/ride-predict/Riders.csv')
#sample_submission = pd.read_csv('../input/samplesubmission/SampleSubmission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
riders.head()

In [ ]:
train = train.merge(riders, on='Rider Id', how='left')
test = test.merge(riders, on='Rider Id', how='left')
del riders

In [ ]:
#Exist only in training set!
del_col = ['Arrival at Destination - Day of Month', 'Arrival at Destination - Weekday (Mo = 1)', 'Arrival at Destination - Time']
train.drop(del_col, axis=1, inplace=True)

In [ ]:
#We had 21201 diferrent values for "Order No" in the training set, 
#the same in the test set. (the same size as the number of row)
#so, it will not improve our model, we need to delate them.
print(len(set(train['Order No'].to_list())))
print(len(set(test['Order No'].to_list())))
#There is only one type of Vehicle "Bike", we need to delate it too.
print(len(set(test['Vehicle Type'].to_list())))

#Columns delate 
del train['Order No']; del test['Order No']; del train['Vehicle Type']; del test['Vehicle Type']

In [ ]:
print('We have {} training rows and {} test rows.'.format(train.shape[0], test.shape[0]))
print('We have {} training columns and {} test columns.'.format(train.shape[1], test.shape[1]))

In [ ]:
train.head(n=2)

In [ ]:
#train[train['Precipitation in millimeters']<5]

In [ ]:
test.head(n=2)

# Data Processing

**Outliers**

Let's explore if there is some outliers

In [ ]:
plt.hist(train['Time from Pickup to Arrival'].values, bins=100)
plt.xlabel('Time from Pickup to Arrival')
plt.ylabel('number of train records')
plt.show()

In [ ]:
plt.hist(train['Average_Rating'].values, bins=100)
plt.xlabel('Average_Rating')
plt.ylabel('number of train records')
plt.show()

In [ ]:
len(set(list(train['Average_Rating'].to_list())))

In [ ]:
plt.hist(train['Platform Type'].values, bins=100)
plt.xlabel('Platform Type')
plt.ylabel('number of train records')
plt.show()

In [ ]:
plt.hist(train['No_Of_Orders'].values, bins=100)
plt.xlabel('No_Of_Orders')
plt.ylabel('number of train records')
plt.show()

In [ ]:
train.describe()

In [ ]:
train['Placement - Time'] = pd.to_datetime(train['Placement - Time'])
train['Confirmation - Time']  = pd.to_datetime(train['Confirmation - Time'])
train['Arrival at Pickup - Time']  = pd.to_datetime(train['Arrival at Pickup - Time'])
train['Pickup - Time']  = pd.to_datetime(train['Pickup - Time'])

In [ ]:
test['Placement - Time'] = pd.to_datetime(test['Placement - Time'])
test['Confirmation - Time']  = pd.to_datetime(test['Confirmation - Time'])
test['Arrival at Pickup - Time']  = pd.to_datetime(test['Arrival at Pickup - Time'])
test['Pickup - Time']  = pd.to_datetime(test['Pickup - Time'])

We can calulate the distance in a sphere when latitudes and longitudes are given by Haversine formula

**haversine(θ) = sin²(θ/2)**

In [ ]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [train, test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [ ]:
haversine_distance('Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long')

In [ ]:
train.head(n=1)

In [ ]:
data = [train,test]
for i in data:
    i['Minute_placement'] = i['Placement - Time'].dt.minute
    i['Hour_placement'] = i['Placement - Time'].dt.hour
    i['Minute_confirmation'] = i['Confirmation - Time'].dt.minute
    i['Hour_confirmation'] = i['Confirmation - Time'].dt.hour
    i['Minute_arriv'] = i['Arrival at Pickup - Time'].dt.minute
    i['Hour_arriv'] = i['Arrival at Pickup - Time'].dt.hour
    i['Minute_pickup'] = i['Pickup - Time'].dt.minute
    i['Hour_pickup'] = i['Pickup - Time'].dt.hour

In [ ]:
del train['Placement - Time']; del train['Confirmation - Time']; del train['Arrival at Pickup - Time']; del train['Pickup - Time'];
del test['Placement - Time']; del test['Confirmation - Time']; del test['Arrival at Pickup - Time']; del test['Pickup - Time'];

In [ ]:
print('We have {} training rows and {} test rows.'.format(train.shape[0], test.shape[0]))
print('We have {} training columns and {} test columns.'.format(train.shape[1], test.shape[1]))

# Feature Egineering

In [ ]:
train.head(n=1)

In [ ]:
train['rating/orders'] = train['No_of_Ratings'] / train['No_Of_Orders']
test['rating/orders'] = test['No_of_Ratings'] / test['No_Of_Orders']

train['int_avg_rating'] = train['Average_Rating'].apply(np.int64)
test['int_avg_rating'] = test['Average_Rating'].apply(np.int64)

train['Rider_number_len'] = train['Rider Id'].apply(lambda x : len(x.split('_')[2]) )
test['Rider_number_len'] = test['Rider Id'].apply(lambda x : len(x.split('_')[2]) )

train['User_number_len'] = train['User Id'].apply(lambda x : len(x.split('_')[2]) )
test['User_number_len'] = test['User Id'].apply(lambda x : len(x.split('_')[2]) )


train['interaction_Rider_User'] = train['Rider Id'] + '_' + train['User Id']
test['interaction_Rider_User'] = test['Rider Id'] + '_' + test['User Id']

train['interaction_User_Platform_Type'] = train['User Id'] + '_' + train['Platform Type'].astype(str)
test['interaction_User_Platform_Type'] = test['User Id'] + '_' + test['Platform Type'].astype(str)

train['interaction_Rider_Average_Rating'] = train['Rider Id'] + '_' + train['Average_Rating'].astype(str)
test['interaction_Rider_Average_Rating'] = test['Rider Id'] + '_' + test['Average_Rating'].astype(str)

train['interaction_Rider_Precipitation'] = train['Rider Id'] + '_' + train['Precipitation in millimeters'].astype(str)
test['interaction_Rider_Precipitation'] = test['Rider Id'] + '_' + test['Precipitation in millimeters'].astype(str)

train['interaction_Temperature_Precipitation'] = train['Temperature'].astype(str) + '_' + train['Precipitation in millimeters'].astype(str)
test['interaction_Temperature_Precipitation'] = test['Temperature'].astype(str) + '_' + test['Precipitation in millimeters'].astype(str)

In [ ]:
train['diff_KM_Harv'] = train['Distance (KM)'] - train['H_Distance']
test['diff_KM_Harv'] = test['Distance (KM)'] - test['H_Distance']

# Create absolute differences
train['abs_lat_diff'] = (train['Destination Lat'] - train['Pickup Lat']).abs()
train['abs_lon_diff'] = (train['Destination Long'] - train['Pickup Long']).abs()
test['abs_lat_diff'] = (test['Destination Lat'] - test['Pickup Lat']).abs()
test['abs_lon_diff'] = (test['Destination Long'] - test['Pickup Long']).abs()

In [ ]:
def minkowski_distance(x1, x2, y1, y2, p):
    return ((abs(x2 - x1) ** p) + (abs(y2 - y1)) ** p) ** (1 / p)
train['manhattan'] = minkowski_distance(train['Pickup Long'], train['Destination Long'],
                                       train['Pickup Lat'], train['Destination Lat'], 1)

test['manhattan'] = minkowski_distance(test['Pickup Long'], test['Destination Long'],
                                       test['Pickup Lat'], test['Destination Lat'], 1)

In [ ]:
# Absolute difference in latitude and longitude
train['abs_lat_diff'] = (train['Destination Lat'] - train['Pickup Lat']).abs()
train['abs_lon_diff'] = (train['Destination Long'] - train['Pickup Long']).abs()

test['abs_lat_diff'] = (test['Destination Lat'] - test['Pickup Lat']).abs()
test['abs_lon_diff'] = (test['Destination Long'] - test['Pickup Long']).abs()


In [ ]:
train['delta_lat'] = train['Destination Lat'] - train['Pickup Lat']
train['delta_lon'] = train['Destination Long'] - train['Pickup Long']

test['delta_lat'] = test['Destination Lat'] - test['Pickup Lat']
test['delta_lon'] = test['Destination Long'] - test['Pickup Long']

In [ ]:
def calculate_direction(d_lon, d_lat):
    result = np.zeros(len(d_lon))
    l = np.sqrt(d_lon**2 + d_lat**2)
    result[d_lon>0] = (180/np.pi)*np.arcsin(d_lat[d_lon>0]/l[d_lon>0])
    idx = (d_lon<0) & (d_lat>0)
    result[idx] = 180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    idx = (d_lon<0) & (d_lat<0)
    result[idx] = -180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    return result

train['direction'] = calculate_direction(train['delta_lon'], train['delta_lat'])
test['direction'] = calculate_direction(test['delta_lon'], test['delta_lat'])

In [ ]:
# plot histogram of directions
plt.figure(figsize=(10,6))
train['direction'].hist(bins=180)
plt.xlabel('direction (degrees)')
plt.title('Histogram direction (Manhattan)')



In [ ]:
#len(set(train['direction'].to_list()))
li=[]
l = train['direction'].to_list()
for i in l:
    li.append(int(i))
len(set(li))

In [ ]:
train['direction_sign'] = np.where(train['direction']>=0, 1, 0)
test['direction_sign'] = np.where(test['direction']>=0, 1, 0)

In [ ]:
train['direction_int'] = train['direction'].apply(lambda x: int(x))
test['direction_int'] = test['direction'].apply(lambda x: int(x))

In [ ]:
train.columns.values

In [ ]:
train['pickup_week_hour'] = train['Pickup - Weekday (Mo = 1)'] * 24 + train['Hour_pickup']
test['pickup_week_hour'] = test['Pickup - Weekday (Mo = 1)'] * 24 + test['Hour_pickup']

In [ ]:
train['pickup_lat_bin'] = np.round(train['Pickup Lat'], 2)
train['pickup_long_bin'] = np.round(train['Pickup Long'], 2)
train['center_lat_bin'] = np.round(train['Pickup Lat'], 2)
train['center_long_bin'] = np.round(train['Pickup Long'], 2)
#train['pickup_dt_bin'] = (train['pickup_dt'] // (3 * 3600))
test['pickup_lat_bin'] = np.round(test['Pickup Lat'], 2)
test['pickup_long_bin'] = np.round(test['Pickup Long'], 2)
test['center_lat_bin'] = np.round(test['Pickup Lat'], 2)
test['center_long_bin'] = np.round(test['Pickup Long'], 2)
#test['pickup_dt_bin'] = (test['pickup_dt'] // (3 * 3600))

In [ ]:
#train['avg_speed_h'] = 1000 * train['H_Distance'] / train['Time from Pickup to Arrival']
#train['avg_speed_km'] = 1000 * train['Distance (KM)'] / train['Time from Pickup to Arrival']
#test['avg_speed_h'] = 1000 * test['H_Distance'] / test['Time from Pickup to Arrival']
#test['avg_speed_km'] = 1000 * test['Distance (KM)'] / test['Time from Pickup to Arrival']

In [ ]:
train.describe()

# Map

In [ ]:
city_long_border = (36.7, 37.2)
city_lat_border = (-1.44, -1.15)
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True)
ax[0].scatter(train['Pickup Long'].values, train['Pickup Lat'].values,
              color='blue', s=1, label='train', alpha=0.1)
ax[1].scatter(test['Pickup Long'].values, test['Pickup Lat'].values,
              color='green', s=1, label='test', alpha=0.1)
fig.suptitle('Train and test area complete overlap.')
ax[0].legend(loc=0)
ax[0].set_ylabel('latitude')
ax[0].set_xlabel('longitude')
ax[1].set_xlabel('longitude')
ax[1].legend(loc=0)
plt.ylim(city_lat_border)
plt.xlim(city_long_border)
plt.show()

In [ ]:
coords = np.vstack((train[['Pickup Lat', 'Pickup Long']].values,
                    train[['Destination Lat', 'Destination Long']].values,
                    test[['Pickup Long', 'Pickup Long']].values,
                    test[['Destination Lat', 'Destination Long']].values))

pca = PCA().fit(coords)
train['pickup_pca0'] = pca.transform(train[['Pickup Lat', 'Pickup Long']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['Pickup Lat', 'Pickup Long']])[:, 1]
train['dropoff_pca0'] = pca.transform(train[['Destination Lat', 'Destination Long']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['Destination Lat', 'Destination Long']])[:, 1]
test['pickup_pca0'] = pca.transform(test[['Pickup Lat', 'Pickup Long']])[:, 0]
test['pickup_pca1'] = pca.transform(test[['Pickup Lat', 'Pickup Long']])[:, 1]
test['dropoff_pca0'] = pca.transform(test[['Destination Lat', 'Destination Long']])[:, 0]
test['dropoff_pca1'] = pca.transform(test[['Destination Lat', 'Destination Long']])[:, 1]

In [ ]:
train['center_latitude'] = (train['Pickup Lat'].values + train['Destination Lat'].values) / 2
train['center_longitude'] = (train['Pickup Long'].values + train['Destination Long'].values) / 2
test['center_latitude'] = (test['Pickup Lat'].values + test['Destination Lat'].values) / 2
test['center_longitude'] = (test['Pickup Long'].values + test['Destination Long'].values) / 2

# Clustring

In [ ]:
sample_ind = np.random.permutation(len(coords))
kmeans = MiniBatchKMeans(n_clusters=1000, batch_size=10000).fit(coords[sample_ind])

In [ ]:
train['pickup_cluster'] = kmeans.predict(train[['Pickup Lat', 'Pickup Long']])
train['dropoff_cluster'] = kmeans.predict(train[['Pickup Lat', 'Pickup Long']])
test['pickup_cluster'] = kmeans.predict(test[['Pickup Lat', 'Pickup Long']])
test['dropoff_cluster'] = kmeans.predict(test[['Pickup Lat', 'Pickup Long']])
t1 = dt.datetime.now()
print('Time till clustering: %i seconds' % (t1 - t0).seconds)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)
ax.scatter(train['Pickup Long'].values, train['Pickup Lat'].values, s=100, lw=0,
           c=train.pickup_cluster.values, cmap='tab20', alpha=0.2)
ax.set_xlim(city_long_border)
ax.set_ylim(city_lat_border)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()

**Imputing missing values**



I will change NaN values for **Precipitation in millimeters** feature with **0**

Temperature values are **very distrubuted** in the same day, so we will replace **NaN** values with the **average of the column**

In [ ]:
#train['Precipitation in millimeters'] = train['Precipitation in millimeters'].fillna((train['Precipitation in millimeters'].mean()))
#test['Precipitation in millimeters'] = test['Precipitation in millimeters'].fillna((test['Precipitation in millimeters'].mean()))

train['Precipitation in millimeters'] = train.groupby(['Arrival at Pickup - Day of Month', 'Hour_pickup'])['Precipitation in millimeters'].transform(lambda x: x.fillna(x.mean()))
test['Precipitation in millimeters'] = test.groupby(['Arrival at Pickup - Day of Month', 'Hour_pickup'])['Precipitation in millimeters'].transform(lambda x: x.fillna(x.mean()))

#train['Temperature'] = train['Temperature'].fillna((train['Temperature'].mean()))
#test['Temperature'] = test['Temperature'].fillna((test['Temperature'].median()))

train['Temperature'] = train.groupby(['Arrival at Pickup - Day of Month', 'Hour_pickup'])['Temperature'].transform(lambda x: x.fillna(x.mean()))
test['Temperature'] = test.groupby(['Arrival at Pickup - Day of Month', 'Hour_pickup'])['Temperature'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
train['Precipitation_level'] = train['Precipitation in millimeters'].apply(lambda x: 1 if x < 1 else 0)
test['Precipitation_level'] = test['Precipitation in millimeters'].apply(lambda x: 1 if x < 1 else 0)

**Check remaining missing values in Precipitation in millimeters**

In [ ]:
train.head(n=1)

In [ ]:
train.head(n=1)

# Using LabelEncoder

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_features = ['User Id', 'Personal or Business', 'Rider Id', 'interaction_Rider_User', 
                'interaction_User_Platform_Type', 'interaction_Rider_Average_Rating', 
                'Average_Rating', 'Precipitation in millimeters', 
                'interaction_Rider_Precipitation', 'interaction_Temperature_Precipitation', 
                'Rider_number_len', 'User_number_len']


# Create new columns in clicks using preprocessing.LabelEncoder()
for feature in cat_features:
    train[feature+'_labels'] = le.fit_transform(train[feature])
    
for feature in cat_features:
    test[feature+'_labels'] = le.fit_transform(test[feature])


# CountEncoder

In [ ]:
train.head()

In [ ]:
import category_encoders as ce
from category_encoders.count import CountEncoder
count_features = ['Personal or Business_labels', 'interaction_Rider_User', 
                'interaction_User_Platform_Type', 'interaction_Rider_Average_Rating', 
                  'Average_Rating', 'Precipitation in millimeters',
                  'interaction_Rider_Precipitation', 'interaction_Temperature_Precipitation', 
                'Rider_number_len', 'User_number_len', 'direction_int']

# Create the count encoder
count_enc = CountEncoder(cols=cat_features)

# Learn encoding from the training set
count_enc.fit(train[cat_features])


# Apply encoding to the train and validation sets
train = train.join(count_enc.transform(train[cat_features]).add_suffix('_count'))
test = test.join(count_enc.transform(test[cat_features]).add_suffix('_count'))


In [ ]:
train.head()

In [ ]:
len(set(train['Precipitation in millimeters'].to_list()))

# Target encoding


In [ ]:
import category_encoders as ce
cat_features = ['User Id', 'Personal or Business', 'Rider Id', 'interaction_Rider_User', 
                'interaction_User_Platform_Type', 'interaction_Rider_Average_Rating', 
                'Average_Rating', 'Precipitation in millimeters',
               'interaction_Rider_Precipitation', 'interaction_Temperature_Precipitation', 
                'Rider_number_len', 'User_number_len', 'direction_int']
from category_encoders import TargetEncoder

# Create the target encoder. You can find this easily by using tab completion.
# Start typing ce. the press Tab to bring up a list of classes and functions.
target_enc = ce.TargetEncoder(cols=cat_features)


# Learn encoding from the training set. Use the 'is_attributed' column as the target.
target_enc.fit(train[cat_features], train['Time from Pickup to Arrival'])

# Apply encoding to the train and validation sets as new columns
# Make sure to add `_target` as a suffix to the new columns
train = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'), how='left')
test = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'), how='left')

In [ ]:
train.head(n=1)

# CatBoost Encoding


In [ ]:
from category_encoders import CatBoostEncoder

cat_features = ['User Id', 'Personal or Business', 'Rider Id', 'interaction_Rider_User', 
                'interaction_User_Platform_Type', 'interaction_Rider_Average_Rating', 
                'Average_Rating', 'Precipitation in millimeters',
               'interaction_Rider_Precipitation', 'interaction_Temperature_Precipitation', 
                'Rider_number_len', 'User_number_len', 'direction_int']

# Create the CatBoost encoder
cb_enc = ce.CatBoostEncoder(cols=cat_features)


# Learn encoding from the training set
cb_enc.fit(train[cat_features], train['Time from Pickup to Arrival'])


# Apply encoding to the train and validation sets as new columns
# Make sure to add `_cb` as a suffix to the new columns
train = train.join(cb_enc.transform(train[cat_features]).add_suffix('_cb'))
test = test.join(cb_enc.transform(test[cat_features]).add_suffix('_cb'))


# Drop Catecory Columns

In [ ]:
train = train.drop(['User Id', 'Personal or Business', 'Rider Id', 'interaction_Rider_User', 
                'interaction_User_Platform_Type', 'interaction_Rider_Average_Rating', 
                    'interaction_Rider_Precipitation', 'interaction_Temperature_Precipitation'], axis = 1)
test = test.drop(['User Id', 'Personal or Business', 'Rider Id', 'interaction_Rider_User', 
                'interaction_User_Platform_Type', 'interaction_Rider_Average_Rating', 
                  'interaction_Rider_Precipitation', 'interaction_Temperature_Precipitation'], axis = 1)

In [ ]:
plt.hist(train['Personal or Business_labels'].values, bins=100)
plt.xlabel('Personal or Business')
plt.ylabel('number of train records')
plt.show()

In [ ]:
#indexNames = train[ (train['Time from Pickup to Arrival'] < 10)].index
#train.drop(indexNames , inplace=True)

In [ ]:
columns = train.columns.values
columns

In [ ]:
label_count_features = [ 'Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'User Id_labels', 'Personal or Business_labels',
       'Rider Id_labels', 'User Id_count', 'Personal or Business_count',
       'Rider Id_count']

In [ ]:
cast_features = ['Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long',  'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'User Id_cb', 'Personal or Business_cb',
       'Rider Id_cb']

In [ ]:
all_features = ['Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 
        'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'User Id_labels', 'Personal or Business_labels',
       'Rider Id_labels', 'User Id_count', 'Personal or Business_count',
       'Rider Id_count', 'User Id_target', 'Personal or Business_target',
       'Rider Id_target']

In [ ]:
features_target = [ 'Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 
        'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'User Id_target', 'Personal or Business_target',
       'Rider Id_target']

In [ ]:
features_count = [ 'Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 
        'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'User Id_labels', 'Personal or Business_labels',
       'Rider Id_labels', 'User Id_count', 'Personal or Business_count',
      ]

In [ ]:
features = ['Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'User Id_labels', 'Personal or Business_labels',
       'Rider Id_labels']

In [ ]:
new_features = ['Platform Type', 
        'Placement - Weekday (Mo = 1)',
        'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long',  'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup',  'abs_lat_diff',
       'abs_lon_diff', 'delta_lat', 'delta_lon', 'direction',
       'User Id_labels', 'Personal or Business_labels', 'Rider Id_labels',
       'interaction_labels', 'center_latitude', 'center_longitude']

In [ ]:
ff = ['Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long',  'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'diff_KM_Harv', 'abs_lat_diff',
       'abs_lon_diff', 'manhattan', 'delta_lat', 'delta_lon', 'direction',
       'center_latitude', 'center_longitude', 'User Id_labels',
       'Personal or Business_labels', 'Rider Id_labels',
       'interaction_labels', 'User Id_count',
       'Personal or Business_count', 'Rider Id_count',
       'interaction_count']

In [ ]:
alll = [
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
        'Confirmation - Weekday (Mo = 1)',
       
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long',  'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
        'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup',  'abs_lat_diff',
       'abs_lon_diff', 'manhattan', 'delta_lat', 'delta_lon',
       'pickup_week_hour', 'pickup_lat_bin', 'pickup_long_bin',
       'center_latitude',
       'center_longitude', 'pickup_cluster', 'dropoff_cluster',
       'User Id_labels', 'Personal or Business_labels', 'Rider Id_labels',
       'interaction_labels']

In [ ]:
# 723

many = ['pickup_long_bin',
 'Hour_pickup',
 'interaction_Rider_Average_Rating_count',
 'Arrival at Pickup - Day of Month',
 'Pickup Lat',
 'Hour_placement',
 'Hour_confirmation',
 'pickup_cluster',
 'Destination Lat',
 'interaction_Rider_User_count',
 'Pickup Long',
 'interaction_Rider_User_labels',
 'Hour_arriv',
 'Rider Id_labels',
 'interaction_User_Platform_Type_count',
 'Precipitation in millimeters',
 'Destination Long',
 'User Id_labels',
 'Average_Rating',
 'pickup_pca0',
 'delta_lat',
 'Pickup - Day of Month',
 'interaction_Rider_Average_Rating_labels',
 'direction',
 'Rider Id_count',
 'pickup_pca1',
 'No_Of_Orders',
 'H_Distance',
 'User Id_count',
 'Average_Rating_target',
 'delta_lon',
 'interaction_User_Platform_Type_labels',
 'manhattan',
 'Minute_placement',
 'abs_lon_diff',
 'Minute_confirmation',
 'No_of_Ratings',
 'pickup_week_hour',
 'dropoff_pca1',
 'Distance (KM)',
 'dropoff_pca0',
 'center_longitude',
 'Minute_arriv',
 'dropoff_cluster',
 'Minute_pickup',
 'abs_lat_diff',
 'Temperature',
 'Age',
 'diff_KM_Harv',
 'center_latitude',
 'Rider Id_target']

In [ ]:
plt.figure(1)
corr = train[many].corr()
sns.heatmap(corr)

# LGBM Model

In [ ]:
#feature_cols = new_features
feature_cols = many

valid_fraction = 0.1
train_srt = train #.sort_values('click_time')
valid_rows = int(len(train_srt) * valid_fraction)
train = train_srt[:-valid_rows * 2]
# valid size == test size, last two sections of the data
valid = train_srt[-valid_rows * 2:]
test = test

In [ ]:
import lightgbm as lgb

dtrain = lgb.Dataset(train[feature_cols], label=train['Time from Pickup to Arrival'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['Time from Pickup to Arrival'])
#test = lgb.Dataset(test[feature_cols], label=test['Time from Pickup to Arrival'])

param = {'num_leaves': 31, 'objective': 'regression', 'learning_rate': 0.03,
              "boosting": "gbdt", "feature_fraction": 0.7}
param['metric'] = 'rmse'
param["max_depth"] = 5
param["verbosity"] = 0
#param["nthread"] = -1
param["min_child_weight"] = 1
#param["bagging_fraction"] = 0.8
param["bagging_seed"] = 2017

num_round = 5000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=50)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(bst.feature_importance(),train[many].columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
feature_imp

In [ ]:
many = feature_imp['Feature'].to_list()
many[5:]

In [ ]:
ypred = bst.predict(test[feature_cols])
sample_submission = pd.read_csv('../input/samplesubmission/SampleSubmission.csv')
sample_submission['Time from Pickup to Arrival'] = ypred
sample_submission.to_csv('submission1.csv', index=False, float_format='%.4f')


In [ ]:
sample_submission.head(n=20)

In [ ]:
featuress = many
y = train['Time from Pickup to Arrival'].values
Xtr, Xv, ytr, yv = train_test_split(train[featuress].values, y, test_size=0.2, random_state=1987)
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(test[featuress].values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

# Try different parameters! My favorite is random search :)
xgb_pars = {'min_child_weight': 1, 'eta': 0.3, 'colsample_bytree': 0.3, 'max_depth': 3,
            'subsample': 0.8, 'lambda': 0.03, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

In [ ]:
# You could try to train with more epoch
model = xgb.train(xgb_pars, dtrain, 1000, watchlist, early_stopping_rounds=500,
                  maximize=False, verbose_eval=10)

In [ ]:
print('Modeling RMSLE %.5f' % model.best_score)
t1 = dt.datetime.now()
print('Training time: %i seconds' % (t1 - t0).seconds)

In [ ]:
feature_importance_dict = model.get_fscore()
fs = ['f%i' % i for i in range(len(featuress))]
f1 = pd.DataFrame({'f': list(feature_importance_dict.keys()),
                   'importance': list(feature_importance_dict.values())})
f2 = pd.DataFrame({'f': fs, 'feature_name': featuress})
feature_importance = pd.merge(f1, f2, how='right', on='f')
feature_importance = feature_importance.fillna(0)

feature_importance[['feature_name', 'importance']].sort_values(by='importance', ascending=False)

# 723

many = ['Platform Type', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Confirmation - Day of Month',
       'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long',
       'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'diff_KM_Harv', 'abs_lat_diff', 'abs_lon_diff',
       'manhattan', 'delta_lat', 'delta_lon', 'pickup_week_hour',
       'pickup_lat_bin', 'pickup_long_bin', 'center_lat_bin',
       'center_long_bin', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1', 'center_latitude', 'center_longitude',
       'pickup_cluster', 'dropoff_cluster', 'User Id_labels',
       'Personal or Business_labels', 'Rider Id_labels',
       'interaction_Rider_User_labels',
       'interaction_User_Platform_Type_labels',
       'interaction_Rider_Average_Rating_labels', 'User Id_count',
       'Personal or Business_count', 'Rider Id_count',
       'interaction_Rider_User_count',
       'interaction_User_Platform_Type_count',
       'interaction_Rider_Average_Rating_count',
       'Rider Id_target',
       'Average_Rating_target']

# 725
many = ['Platform Type', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Confirmation - Day of Month',
       'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long',
       'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'diff_KM_Harv', 'abs_lat_diff', 'abs_lon_diff',
       'manhattan', 'delta_lat', 'delta_lon', 'pickup_week_hour',
       'pickup_lat_bin', 'pickup_long_bin', 'center_lat_bin',
       'center_long_bin', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1', 'center_latitude', 'center_longitude',
       'pickup_cluster', 'dropoff_cluster', 'User Id_labels',
       'Personal or Business_labels', 'Rider Id_labels',
       'interaction_Rider_User_labels',
       'interaction_User_Platform_Type_labels',
       'interaction_Rider_Average_Rating_labels', 'User Id_count',
       'Personal or Business_count', 'Rider Id_count',
       'interaction_Rider_User_count',
       'interaction_User_Platform_Type_count',
       'interaction_Rider_Average_Rating_count',
       'Rider Id_target',
       'Average_Rating_target']

# 726
many = ['Platform Type', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Confirmation - Day of Month',
       'Confirmation - Weekday (Mo = 1)',
       'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Distance (KM)', 'Temperature',
       'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long',
       'No_Of_Orders', 'Age',
       'Average_Rating', 'No_of_Ratings', 'H_Distance',
       'Minute_placement', 'Hour_placement', 'Minute_confirmation',
       'Hour_confirmation', 'Minute_arriv', 'Hour_arriv', 'Minute_pickup',
       'Hour_pickup', 'diff_KM_Harv', 'abs_lat_diff', 'abs_lon_diff',
       'manhattan', 'delta_lat', 'delta_lon', 'pickup_week_hour',
       'pickup_lat_bin', 'pickup_long_bin', 'center_lat_bin',
       'center_long_bin', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1', 'center_latitude', 'center_longitude',
       'pickup_cluster', 'dropoff_cluster', 'User Id_labels',
       'Personal or Business_labels', 'Rider Id_labels',
       'interaction_Rider_User_labels',
       'interaction_User_Platform_Type_labels',
       'interaction_Rider_Average_Rating_labels', 'User Id_count',
       'Personal or Business_count', 'Rider Id_count',
       'interaction_Rider_User_count',
       'interaction_User_Platform_Type_count',
       'interaction_Rider_Average_Rating_count',
       'Rider Id_target',
       'Average_Rating_target']